In [2]:
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
import matplotlib.pyplot as plt

from utils.redraw_obstacles import redraw_obstacles, set_non_blocking
from utils.create_draw_params import create_draw_params
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np

from commonroad.prediction.prediction import Prediction, Occupancy
def occupancy_at_time(self, time_step: int):
    """
    Returns the predicted occupancy of the obstacle at a specific time step.

    :param time_step: discrete time step
    :return: predicted occupancy of the obstacle at time step
    """
    occupancy = None
    if time_step == self.initial_state.time_step:
        shape = self.obstacle_shape.rotate_translate_local(
            self.initial_state.position, self.initial_state.orientation)
        occupancy = Occupancy(time_step, shape)
    elif time_step > self.initial_state.time_step and time_step < (self.initial_state.time_step + len(self._prediction.trajectory.state_list)) and   self._prediction is not None:
        occupancy = self._prediction.occupancy_at_time_step(time_step)
    return occupancy

from commonroad.scenario.obstacle import StaticObstacle, DynamicObstacle, Obstacle
DynamicObstacle.occupancy_at_time = occupancy_at_time

In [3]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/highway-ramp.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()

In [4]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    draw_params={'time_begin':0}
    handles = {}
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()

In [5]:
show_lane(scenario)

KeyboardInterrupt: 

In [25]:
def plt_window(sce, dur, draw_params, size, pos):
    set_non_blocking()  # ensures interactive plotting is activated
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    # ax.set_facecolor("black")
    # ax.set_cmap("binary")
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_params['time_begin'] = dur[0]
    draw_object(sce, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
    # plt.gray()
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    data = np.zeros(( (duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(dur[0], dur[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(sce, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
        buffer = buffer[:, : , 0].astype(bool)
        data[(i-dur[0]), :, : ] = buffer
        print('fps:', (i - dur[0])/(time.time()-t1))
    return data

In [17]:
size = [200, 40]
pos = [350, 135]
duration = [0, 100]
inch_in_cm = 10
draw_params = create_draw_params()

origin_data = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 8.877190828837561
fps: 13.09310445473545
fps: 16.06165841005957
fps: 18.16750227943613
fps: 20.02211151612823
fps: 20.95615215508627
fps: 21.79795845487597
fps: 22.51685151980416
fps: 23.225074814931485
fps: 23.711154175777708
fps: 24.33596750797795
fps: 24.514378895237947
fps: 24.259363550414328
fps: 23.140969347019233
fps: 22.949800448603067
fps: 23.108030753380696
fps: 23.42159828717874
fps: 23.47149501330614
fps: 23.54943305611874
fps: 23.870537048155917
fps: 24.0173879895587
fps: 24.168134831802046
fps: 24.298404756663302
fps: 24.39351317608845
fps: 24.528718977863996
fps: 24.75607665373607
fps: 25.052248240808726
fps: 25.311573463913817
fps: 25.482529665765327
fps: 25.53455834321097
fps: 25.772750662240366
fps: 25.787406878833256
fps: 25.869082050753942
fps: 25.933696340456116
fps: 26.029714527265742
fps: 25.97347795483147
fps: 25.992449943062205
fps: 25.995024228101336
fps: 26.018994846873554
fps: 26.066598314649948
fps: 26.11097506462208
fps: 25.488737169958032
fp

In [89]:
def plt_data(data):
    fig = plt.figure()
    plt.gray()
    if len(data.shape) <=2:
        plt.imshow(data)
    elif len(data.shape) > 3:
        for r in data:
            myobj = plt.imshow(r[0, :, :])
            for rr in r:
                myobj.set_data(rr)
                plt.draw()
                plt.pause(0.01)
    else:
        myobj = plt.imshow(data[0, :, :])
        for row in data:
            myobj.set_data(row)
            plt.draw()
            plt.pause(0.01)

In [103]:
plt_data(data)

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/_backend_tk.py", line 346, in idle_draw
    self.draw()
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/backend_agg.py", line 402, in draw
    self.figure.draw(self.renderer)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/artist.py", line 50, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/

KeyboardInterrupt: 

In [48]:
from commonroad.scenario.scenario import Scenario

ego_scenario = Scenario(dt=0.1, benchmark_id=scenario.benchmark_id)
ego_scenario.lanelet_network = scenario.lanelet_network

In [110]:
leng = 100
start_goal = np.zeros((leng, 6), dtype=np.dtype(float))

ego_id = 0
set_non_blocking()  # ensures interactive plotting is activated
figsize = size
fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
# ax.set_facecolor("black")
# ax.set_cmap("binary")
fig.add_axes(ax)
fig.patch.set_facecolor('black')
plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
plt.set_cmap("plasma")
plt.gca().set_aspect('equal')
handles = {}  # collects handles of obstacle patches, plotted by matplotlib
draw_object(scenario.lanelet_network, handles=handles, draw_params=draw_params)
# plt.gray()
fig.canvas.draw()
for handles_i in handles:
    if not handles_i:
        handles.pop()

data = np.zeros(( (leng, duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

for index in range(0, leng):
    ego = None
    while(scenario.obstacle_by_id(ego_id) == None):
        ego_id = ego_id + 1
    ego = scenario.obstacle_by_id(ego_id)
    ego_id = ego_id + 1
    print(ego_id)
    size = [400, 30]
    scenario.remove_obstacle(ego)
    traj = ego.prediction.trajectory
    initial_time = traj.initial_time_step
    t = 50
    start_time = initial_time + t
    end_time = start_time + 50
    pos = traj.state_list[t].position + [100, 0]
    start_goal[index, 0:2] = traj.state_list[t].position
    start_goal[index, 2] = traj.state_list[t].orientation
    start_goal[index, 3:5] = traj.state_list[t+50].position
    start_goal[index, 5] = traj.state_list[t+50].orientation
    # ego_scenario.add_objects(ego)
    # ego_data = plt_window(sce=ego_scenario, dur=[start_time, end_time], draw_params=draw_params, size=size, pos=pos)
    # ego_scenario.remove_obstacle(ego)
    # data = plt_window(sce=scenario, dur=[start_time, end_time], draw_params=draw_params, size=size, pos=pos)

    duration = [start_time, end_time]

    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(duration[0], duration[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
        buffer = buffer[:, : , 0].astype(bool)
        data[index, (i-duration[0]), :, : ] = buffer
        print('fps:', (i - duration[0])/(time.time()-t1), " time:", i)
    scenario.add_objects(ego)

/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/commonroad/scenario/scenario.py:183: UserWarning: <Scenario/obstacle_by_id> Obstacle with ID 125 is not contained in the scenario.
  % obstacle_id)


127
fps: 0.0  time: 24300
fps: 7.122317013530402  time: 24301
fps: 9.262302467118339  time: 24302
fps: 9.841979704245777  time: 24303
fps: 10.446773945482203  time: 24304
fps: 10.797288774362288  time: 24305
fps: 10.81388945228036  time: 24306
fps: 10.915426263010808  time: 24307
fps: 10.99305153995422  time: 24308
fps: 11.502918334772643  time: 24309
fps: 12.126422525177706  time: 24310
fps: 12.710026126750552  time: 24311
fps: 13.190285055972359  time: 24312
fps: 13.527528940095367  time: 24313
fps: 13.781347853127897  time: 24314
fps: 14.134498279652268  time: 24315
fps: 14.512403113850876  time: 24316
fps: 14.784550768679543  time: 24317
fps: 15.041029916713734  time: 24318
fps: 15.153601690685164  time: 24319
fps: 15.407014716048803  time: 24320
fps: 15.627704563152736  time: 24321
fps: 15.808927795572696  time: 24322
fps: 15.886073579291471  time: 24323
fps: 16.084106288398427  time: 24324
fps: 16.160570019129274  time: 24325
fps: 16.341973923130674  time: 24326
fps: 16.519988761

KeyboardInterrupt: 

In [105]:
plt_data(data)

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/_backend_tk.py", line 346, in idle_draw
    self.draw()
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/backends/backend_agg.py", line 402, in draw
    self.figure.draw(self.renderer)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/artist.py", line 50, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/

KeyboardInterrupt: 

In [54]:
ego_scenario.add_objects(ego)
ego_data = plt_window(sce=ego_scenario, dur=[start_time, end_time], draw_params=draw_params, size=size, pos=pos)
ego_scenario.remove_obstacle(ego)

fps: 0.0
fps: 10.18274690885961
fps: 11.332556989659858
fps: 12.531445353931403
fps: 13.40661795905908
fps: 13.582397142009635
fps: 13.50215201106534
fps: 13.357540585408952
fps: 13.4306316446328
fps: 13.284029809305325
fps: 13.114232507181976
fps: 13.476549337408661
fps: 13.63517972491052
fps: 13.782084841058243
fps: 13.830641331956556
fps: 13.438366136148487
fps: 13.853165928272785
fps: 14.02981508226655
fps: 13.794067950209604
fps: 13.901453122148991
fps: 13.972562261925246
fps: 14.089797754376288
fps: 14.208169675144173
fps: 14.401452252424402
fps: 14.50071492839048
fps: 14.678660720035165
fps: 14.796415327380407
fps: 14.929623157102071
fps: 15.036872057442034
fps: 15.1808368836672
fps: 15.336697281364605
fps: 15.185034737684324
fps: 15.034051549456505
fps: 15.057907744172287
fps: 14.951491171987561
fps: 14.907633497631393
fps: 14.876089747006382
fps: 14.91791483087661
fps: 14.928563293150061
fps: 14.807521066459648
fps: 14.89775540237585
fps: 14.996887887738612
fps: 15.03270110246

In [56]:
origin_data = plt_window(sce=scenario, dur=[start_time, end_time], draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 5.469751622940516
fps: 7.258201224840838
fps: 8.395240239789434
fps: 8.909639248428473
fps: 9.47173499491445
fps: 9.089243224735512
fps: 9.30962222630908
fps: 9.631243012530444
fps: 9.407864390626612
fps: 9.860635172597979
fps: 9.694870122447332
fps: 10.133586211686966
fps: 10.375991981748468
fps: 10.599932631512061
fps: 10.717877063565641
fps: 10.631455795857384
fps: 10.832549742818134
fps: 10.922963761167114
fps: 11.004483857849374
fps: 11.133157270195866
fps: 11.076020934191897
fps: 11.116646087800778
fps: 11.188759487986253
fps: 11.198935012369489
fps: 11.136714315816663
fps: 11.2222446217264
fps: 10.975563845466253
fps: 11.089249848189171
fps: 11.135361470208663
fps: 11.231112801041093
fps: 11.316294817835688
fps: 11.301313077858035
fps: 11.287692396018727
fps: 11.226864298984722
fps: 11.298547658074497
fps: 11.42936781292935
fps: 11.444032224261937
fps: 11.527205892297449
fps: 11.55699163000864
fps: 11.558501648358765
fps: 11.570718006175662
fps: 11.484559118003265


In [68]:
print(scenario.obstacle_by_id(119))

Dynamic Obstacle:

id: 119
initial state: 
position= [  4.85 139.82]
orientation= -0.0
velocity= 30.0
time_step= 24106



In [39]:
print(scenario.dynamic_obstacles[ego.obstacle_id])

Dynamic Obstacle:

id: 235
initial state: 
position= [  4.85 139.82]
orientation= -0.0
velocity= 30.0
time_step= 26626



In [32]:
print(ego.obstacle_id)

117


In [34]:
print(scenario.dynamic_obstacles[0])

Dynamic Obstacle:

id: 118
initial state: 
position= [  4.85 133.42]
orientation= -0.0
velocity= 30.0
time_step= 24082



In [94]:
obstacle = scenario.obstacle_by_id(ego_id)
scenario.remove_obstacle(scenario.obstacle_by_id(ego_id))
ego_scenario.add_objects(obstacle)
# show_lane(ego_scenario)

In [95]:
ego_scenario.remove_obstacle(ego_scenario.dynamic_obstacles[0])

In [121]:
size = [400, 30]
duration = [0, 0]
tra = ego_scenario.dynamic_obstacles[0].prediction.trajectory
duration[0] = tra.initial_time_step + 40
duration[1] =  tra.initial_time_step + 80
pos = tra.state_list[39].position + [100, 0]
ego_data = plt_window(sce=ego_scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 7.140273911970242
fps: 9.450642896815427
fps: 9.751013041480261
fps: 11.434657942714988
fps: 13.102024700243405
fps: 13.860282571974452
fps: 14.87311705943142
fps: 15.690199012419573
fps: 15.96962158978993
fps: 16.033247655584624
fps: 15.879575363833235
fps: 15.507455227656038
fps: 15.00723499860045
fps: 14.58147138125564
fps: 14.069664727485137
fps: 14.014270950081746
fps: 13.832844581607054
fps: 13.751132593952978
fps: 13.626010343841775
fps: 13.615814495538823
fps: 13.66898299671592
fps: 13.551506546380088
fps: 13.770936696139758
fps: 13.851467483692103
fps: 13.868181806158429
fps: 14.044373262916508
fps: 13.953234338648759
fps: 13.93199096939495
fps: 13.688527267090146
fps: 13.513468349395032
fps: 13.625886112515081
fps: 13.589411783764518
fps: 13.564688383765773
fps: 13.595332413035687
fps: 13.75084900350336
fps: 13.843555796944726
fps: 13.96110555147129
fps: 13.950581485930515
fps: 13.989428512638629


In [167]:
obsereved = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 5.241458254448481
fps: 6.976298272518305
fps: 7.436739986512946
fps: 8.052655249325754
fps: 8.010658713375951
fps: 8.260089271032014
fps: 8.203916738431998
fps: 8.456903841195535
fps: 8.596321880751473
fps: 8.73895443092674
fps: 8.605929629138437
fps: 8.570304161765538
fps: 8.6703834572131
fps: 8.785721349913018
fps: 8.790618207710144
fps: 8.786764873399573
fps: 8.736685784414151
fps: 8.782266254192654
fps: 8.861169505980516
fps: 8.96716404380807
fps: 9.056906802182409
fps: 9.13643156856451
fps: 9.136451155796351
fps: 9.17680701330876
fps: 9.204014467644507
fps: 9.251751587723017
fps: 9.234649663578399
fps: 9.216557712319716
fps: 9.16590658892043
fps: 9.124736373913326
fps: 9.174675232670184
fps: 9.13436868819888
fps: 9.036746458641089
fps: 9.06544592447484
fps: 9.057731619471545
fps: 9.093727550990419
fps: 9.076490168170006
fps: 9.058498248358049
fps: 9.097803728691238


In [115]:
plt_data(obsereved)

In [120]:
print(ego_scenario.dynamic_obstacles[0])
print(scenario.dynamic_obstacles[0])

Dynamic Obstacle:

id: 115
initial state: 
position= [  4.85 139.82]
orientation= -0.0
velocity= 30.0
time_step= 24010

Dynamic Obstacle:

id: 116
initial state: 
position= [  4.85 136.62]
orientation= -0.0
velocity= 30.0
time_step= 24034



In [118]:
obstacle = scenario.dynamic_obstacles[0]
scenario.remove_obstacle(obstacle)
ego_scenario.add_objects(obstacle)

In [119]:
ego_scenario.remove_obstacle(ego_scenario.dynamic_obstacles[0])

In [164]:
from commonroad.prediction.prediction import Prediction, Occupancy
def occupancy_at_time(self, time_step: int) -> Union[None, Occupancy]:
    """
    Returns the predicted occupancy of the obstacle at a specific time step.

    :param time_step: discrete time step
    :return: predicted occupancy of the obstacle at time step
    """
    occupancy = None
    if time_step == self.initial_state.time_step:
        shape = self.obstacle_shape.rotate_translate_local(
            self.initial_state.position, self.initial_state.orientation)
        occupancy = Occupancy(time_step, shape)
    elif time_step > self.initial_state.time_step and time_step < (self.initial_state.time_step + len(self._prediction.trajectory.state_list)) and   self._prediction is not None:
        occupancy = self._prediction.occupancy_at_time_step(time_step)
    return occupancy

In [165]:
from commonroad.scenario.obstacle import StaticObstacle, DynamicObstacle, Obstacle
DynamicObstacle.occupancy_at_time = occupancy_at_time

In [166]:
ego_scenario.dynamic_obstacles[0].occupancy_at_time(24500)